In [16]:
devtools::install_github("hungms/annotatepip", dependencies = F)

Using GitHub PAT from the git credential store.




-- R CMD build -----------------------------------------------------------------
* checking for file '/private/var/folders/nq/x34jjrzs6937hpz46dbzxqfcpzmck6/T/RtmpDVqR9v/remotes1822558f8b90d/hungms-annotatepip-e46dbca/DESCRIPTION' ... OK
* preparing 'annotatepip':
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted 'LazyData' from DESCRIPTION
* building 'annotatepip_0.0.0.tar.gz'



In [1]:
setwd("/Users/hungm/Documents/development/annotatepip/")
library(annotatepip)

Loading required package: BiocManager

Bioconductor version '3.19' is out-of-date; the current release version '3.20'
  is available with R version '4.4'; see https://bioconductor.org/install

Loading required package: tidyverse

Warning message:
"package 'purrr' was built under R version 4.4.1"
Warning message:
"package 'lubridate' was built under R version 4.4.1"
-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.0.4     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: biomaRt

Loading required package: OmnipathR

Warning message:
"package 'OmnipathR

In [2]:
counts <- read.table("data/GSE224465.tsv", sep = "\t", header = T, row.names = 1)
counts[1:5, 1:5]
dim(counts)

,C.CB_R1,C.CB_R2,C.CB_R3,C.CB_R4,C.CC_R1
,<int>,<int>,<int>,<int>,<int>
0610005C13Rik,9,6,12,5,8
0610006L08Rik,0,0,0,0,0
0610009B22Rik,149,156,151,136,138
0610009E02Rik,3,3,4,0,2
0610009L18Rik,25,30,22,14,33


[1] 55291    29

## Remove selected genes from expression matrix

In [3]:
xy.genes <- get_xy_genes(org = "mouse")
xy.genes %>% head(.)

counts <- counts[!rownames(counts) %in% xy.genes,]
dim(counts)

[1] "Gm29289" "Ddx3y"   "Gm10256" "Sry"     "Kdm5d"   "Gm21677"

[1] 54380    29

In [4]:
mt.genes <- get_mt_genes(org = "mouse")
mt.genes %>% head(.)

counts <- counts[!rownames(counts) %in% mt.genes,]
dim(counts)

[1] "mt-Nd4" "mt-Nd1" "mt-Nd5" "mt-Nd6" "mt-Nd2" "mt-Nd3"

[1] 54367    29

In [5]:
vdj.genes <- get_str_genes(org = "mouse", str = c("tcr", "bcr"))
vdj.genes %>% head(.)

counts <- counts[!rownames(counts) %in% vdj.genes,]
dim(counts)

[1] "Ighv3-4"  "Ighv12-3" "Ighv1-55" "Ighv1-15" "Trdv5"    "Ighv1-22"

[1] 54102    29

## Convert gene orthologues

Convert mouse gene symbols to human.

In [6]:
human_genes <- convert_mouse_to_human(rownames(counts), one.to.many = F)
human_genes %>% head(.)

[1] "MT-ND4" "MT-ND1" "MT-ND5" "MT-ND6" "MT-ND2" "MT-ND3"

Convert human gene symbols to mouse.

In [7]:
mouse_genes <- convert_human_to_mouse(human_genes, one.to.many = F)
mouse_genes %>% head(.)

[1] "mt-Nd4" "mt-Nd1" "mt-Nd5" "mt-Nd6" "mt-Nd2" "mt-Nd3"

## Interoperability with GCT, GMT files

Save expression matrix as GCT file.

In [8]:
write_gct(counts, save_name = "GSE224465.gct", save_dir = "tests/")

Load expression matrix from GCT file.

In [9]:
counts <- read_gct("tests/GSE224465.gct")
counts[1:5, 1:5]
dim(counts)

,C.CB_R1,C.CB_R2,C.CB_R3,C.CB_R4,C.CC_R1
,<int>,<int>,<int>,<int>,<int>
0610005C13Rik,9,6,12,5,8
0610006L08Rik,0,0,0,0,0
0610009B22Rik,149,156,151,136,138
0610009E02Rik,3,3,4,0,2
0610009L18Rik,25,30,22,14,33


[1] 54102    29

Save gene list as GMT file.

In [10]:
gene.list <- list(XY = xy.genes, MT = mt.genes, VDJ = vdj.genes)
gene.df <- convert_list_to_df(gene.list)
gene.df %>% head(.)

,XY,MT,VDJ
,<chr>,<chr>,<chr>
1,Gm29289,mt-Nd4,Ighv3-4
2,Ddx3y,mt-Nd1,Ighv12-3
3,Gm10256,mt-Nd5,Ighv1-55
4,Sry,mt-Nd6,Ighv1-15
5,Kdm5d,mt-Nd2,Trdv5
6,Gm21677,mt-Nd3,Ighv1-22


In [11]:
write_gmt(gene.df, save_name = "string_genes.gmt", save_dir = "tests/")

Load gene list from GMT file.

In [12]:
gmt <- read_gmt("tests/string_genes.gmt")
gmt %>% head(.)

,XY,MT,VDJ
,<chr>,<chr>,<chr>
V3,Gm29289,mt-Nd4,Ighv3-4
V4,Ddx3y,mt-Nd1,Ighv12-3
V5,Gm10256,mt-Nd5,Ighv1-55
V6,Sry,mt-Nd6,Ighv1-15
V7,Kdm5d,mt-Nd2,Trdv5
V8,Gm21677,mt-Nd3,Ighv1-22


## Add gene annotations

In [ ]:
counts <- counts[,1:3]
counts$gene <- rownames(counts)

counts.annotated<- run_annotation(counts, gene_column = "gene", org = "mouse")
counts.annotated %>% filter(!is.na(entity_type)) %>% head(.)

,gene,C.CB_R1,C.CB_R2,C.CB_R3,mouse_chromosome,entity_type,category,parent,consensus_score,is_cs,database_cs,database_tf
,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<lgl>,<chr>,<chr>
1,0610010K14Rik,8,16,18,11,protein,intracellular,intracellular,1,FALSE,"ComPPI, OmniPath",NA
2,0610030E20Rik,1452,1485,1087,6,protein,intracellular,intracellular,1,FALSE,"ComPPI, OmniPath",NA
3,0610040J01Rik,1,2,0,5,protein,intracellular,intracellular,1,FALSE,"OmniPath, ComPPI",NA
4,1110032A03Rik,99,83,50,9,protein,intracellular,intracellular,3,FALSE,"OmniPath, GO_Intercell, ComPPI, UniProt",NA
5,1110032F04Rik,0,0,2,3,protein,"cell_surface, transmembrane","cell_surface, transmembrane, transmembrane_predicted",4,TRUE,"Surfaceome, OmniPath, Ramilowski_location, UniProt, Phobius",NA
6,1110038F14Rik,656,665,589,15,protein,intracellular,intracellular,1,FALSE,"ComPPI, OmniPath",NA
